In [ ]:
# CLASSICAL - Based on 20240109_1.py
#%%
# 必要に応じてpip
# !pip install --upgrade pip
# !pip install numpy scikit-learn tensorflow matplotlib pillow pandas

In [2]:
# 機械学習のライブラリ関連をインポート
import pandas as pd
import numpy as np

# ランダムにシャッフルして，学習・テストに分割するモジュール
from sklearn.model_selection import ShuffleSplit

# 深層学習のライブラリをインポート
import tensorflow as tf
import tensorflow.keras as keras

#表示系のインポートと設定
import matplotlib.pyplot as plt

# ファイル操作
import os

# 画像操作
from PIL import Image

# 時間取得
import datetime

In [73]:
# DIRS
DATASET_NUM = 10000
DIRS_DATASET = "../Training/Datasets_v0108/Dataset" + str(DATASET_NUM) + "/"

In [74]:
# ファイルを取得, 配列に格納, Pythonリスト型をnumpy.ndarray型に変換
file_names = []
    
# フォルダ内のファイルを取得
files = os.listdir(DIRS_DATASET)

# ファイル名を配列に格納
for file in files:
    file_names.append(file)

# Pythonリスト型をnumpy.ndarray型に変換
file_names = np.array(file_names)

In [75]:
file_names

array(['4510016121775.jpeg', '4510040841489.jpeg', '4510041988985.jpeg',
       ..., '4999915370062.jpeg', '4999978425181.jpeg',
       '4999985455584.jpeg'], dtype='<U18')

In [76]:
def convert_to_grayscale(numpy_array):
    # グレーと言わず2値化
    grayscale_array = np.where(numpy_array <= 128, 0, 255)
    # plt.imshow(grayscale_array) # こいつらのせいで処理が重かった。出力系は要注意
    # print(grayscale_array)
    return grayscale_array

In [77]:
### 画像を配列にしてよしなに
X, y = [], []

In [78]:
for file_name in file_names:
    numpy_array = np.array(Image.open(DIRS_DATASET + file_name)) # 画像をnumpy配列にする
    grayscale_array = convert_to_grayscale(numpy_array)
    X.append(grayscale_array)
    y.append(int(file_name[0:13]))

In [79]:
y

[4510016121775,
 4510040841489,
 4510041988985,
 4510043280667,
 4510053336644,
 4510061873322,
 4510068510879,
 4510071401645,
 4510081049288,
 4510100977462,
 4510101743271,
 4510123965798,
 4510181870027,
 4510203751211,
 4510209672695,
 4510237959577,
 4510298890574,
 4510329814852,
 4510334349301,
 4510343785367,
 4510348742914,
 4510350245731,
 4510363047780,
 4510379731741,
 4510383746786,
 4510404406668,
 4510412383869,
 4510425740123,
 4510426309220,
 4510445380767,
 4510454127537,
 4510464464660,
 4510466812810,
 4510493690702,
 4510498257603,
 4510511629059,
 4510521439099,
 4510539297278,
 4510540402401,
 4510553639665,
 4510557019890,
 4510559878013,
 4510566332225,
 4510610633797,
 4510634238770,
 4510659831727,
 4510716787615,
 4510755460494,
 4510771525498,
 4510786030864,
 4510813245407,
 4510835459905,
 4510839100339,
 4510839306632,
 4510887778993,
 4510888210577,
 4510950897576,
 4511001844693,
 4511013956858,
 4511053192711,
 4511117936367,
 4511128893970,
 4511132

In [80]:
# X, yをPythonリスト型をnumpy.ndarray型に変換
X = np.array(X)
X = X.squeeze()
y = np.array(y)

In [81]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [82]:
y

array([4510016121775, 4510040841489, 4510041988985, ..., 4999915370062,
       4999978425181, 4999985455584], dtype=int64)

In [83]:
### データの分割
ss = ShuffleSplit(n_splits=1,      # 分割を1個生成
                  train_size=0.8,  # 学習
                  test_size =0.2,  # テスト
                  random_state=0)  # 乱数種（再現用）

# 学習データとテストデータのインデックスを作成
train_index, test_index = next(ss.split(X))

X_train, X_test = X[train_index], X[test_index] # 学習データ，テストデータ
y_train, y_test = y[train_index], y[test_index] # 学習データのラベル，テストデータのラベル

In [84]:
X_train.shape[0], X_train.shape[1]

(8000, 337)

In [85]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [112]:
### ニューラルネットワークの構築

# 学習し、テストデータで評価し、スコアを表示する
# 引数は、中間層の数、バッチサイズ、epoch数

def fit_epoch(neuron, batch, epochs, ckpt_period, optimizer_name):
    ver_name = "v6_240108_CLASSICAL"
    
    # # チェックポイントの設定
    # dt_now = datetime.datetime.now()
    # checkpoint_path = "./training_ckpt_" + dt_now.strftime('%Y%m%d%H%M%S') + "_" + ver_name + "_d" + str(DATASET_NUM) + "_n" + str(neuron)  + "_b" + str(batch) + "_e" + str(epochs) + "_"+ optimizer_name + "/cp-{epoch:09d}.ckpt"
    # checkpoint_dir = os.path.dirname(checkpoint_path)
    # cp_callback = tf.keras.callbacks.ModelCheckpoint(
    #     checkpoint_path,
    #     verbose=1,
    #     save_weights_only=True,
    #     period=ckpt_period  # 重みをckpt_periodエポックごとに保存します
    # )


    # レイヤーのオブジェクトを作成
    Dense = keras.layers.Dense

    # モデルの構造を定義
    model = keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(337, )))
    model.add(Dense(neuron, activation='relu'))
    
    model.add(Dense(13, activation='linear'))  # 13桁の数字を出力する層

    # モデルを構築
    model.compile(
        loss='mean_squared_error',
        optimizer=optimizer_name,
        metrics=['accuracy']
    )

    # # 必要に応じてチェックポイントから再開
    # model.load_weights("./training_ckpt_20240109084448_v6_240108_d10000_n256_b32_e120000_Adamax/cp-000001000.ckpt")

    # 学習を実行
    hist = model.fit(X_train, y_train,
        batch_size=batch, # 誤差逆伝播法をするときの1回当たりのデータ数
        epochs=epochs,
        # callbacks=[cp_callback],
        verbose=1,
        validation_split=0.1)
    
    # モデルの保存
    dt_now = datetime.datetime.now()
    model.save("./CLASSICAL/" + dt_now.strftime('%Y%m%d%H%M%S') + "_" + ver_name + "_d" + str(DATASET_NUM) + "_n" + str(neuron)  + "_b" + str(batch) + "_e" + str(epochs) + "_"+ optimizer_name)
    
   # モデルを評価
    score = model.evaluate(X_test, y_test, verbose=1)
    print('正解率=', score[1], 'loss=', score[0])
    
     # 学習の様子をグラフへ描画 
    # 正解率の推移をプロット
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('Accuracy')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # ロスの推移をプロット
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Loss')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()   

In [113]:
print(DATASET_NUM)

10000


In [114]:
fit_epoch(4096, 32, 20000, 999999, "Adamax")

Epoch 1/20000
225/225 [==============================] - 2s 9ms/step - loss: 22621747237209040945676288.0000 - accuracy: 0.0000e+00 - val_loss: 22702834512471049707192320.0000 - val_accuracy: 0.0000e+00
Epoch 2/20000
225/225 [==============================] - 2s 8ms/step - loss: 22621675756075755321163776.0000 - accuracy: 0.0000e+00 - val_loss: 22702721526163598236188672.0000 - val_accuracy: 0.0000e+00
Epoch 3/20000
225/225 [==============================] - 2s 8ms/step - loss: 22621514347065110362587136.0000 - accuracy: 0.0000e+00 - val_loss: 22702527835350824285896704.0000 - val_accuracy: 0.0000e+00
Epoch 4/20000
225/225 [==============================] - 2s 8ms/step - loss: 22621279151078170565804032.0000 - accuracy: 0.0000e+00 - val_loss: 22702251134189718642622464.0000 - val_accuracy: 0.0000e+00
Epoch 5/20000
225/225 [==============================] - 2s 8ms/step - loss: 22620970168114935930814464.0000 - accuracy: 0.0000e+00 - val_loss: 22701884505151253665284096.0000 - val_accura

KeyboardInterrupt: 